In [2]:
import torch, transformers, json
from transformers import(
    AutoTokenizer,
    AutoModelForCausalLM,
    pipeline
)

## Dataset value example 
{'record': 'JAHFAU010000168.1', 'text': 'CTCCTCGGCCAGCAAGCGCATCGTGAGAACCGGACTGACGATCAGCAAAACGGTCGCCAGCGGGCCCAATACCATGCGGCCATCCGGGTTCATCTGGCCGGTCGGCAGGCCAAAGTTAACGTTTAGGGAGAAGATCAGGCCGAGCACCAGGAAGCTGAGGGCGGCGGCCGCGTAAGCCAGCGGCGACACGAAGTAGCGGCGGAACTCGCGCACTGCAATAACCCAGCTGGTGCGCACGATCTAGCTCTCCTGCGGCACGACTTGCTCTTCCTCGCGAGTCAGCAGCAGGAAGATGCTTTCCAGGCTGACCCCGGCCGTTGTCAGCTCCAGCAAGTCCAGATCGGCTTCAACGATGCGGCGCGCCAACAGCGGGCGCAGGTCGCGGCCGGGCGAAGAGATCACCTCCACGGTGCCTGCCTGCCCGGCAAGCTCGACCGACTCCACTCCCGGGACGGACTTGAGCAGGTCGACCAGATCGGCTGCCGGGGCAGAGCTGCGGATCAGAATCCGGTACGCACCGGCCAGCTGGGCCGAAAGATTGGCGGGCGTATCTTCCGCCACCAACCGCCCCTTGTTGATAATCAGGACTCGATCACAAACTTGCTGTGCTTCCGAGAGGATGTGGGTGGACAACATGACGGTCCGCTCCCGCCCGAGCTCTCGGATTAAGTCCCGGACTTCGATGATTTGGGCCGGATCCAGTCCAATGGTCGGCTCGTCCAGGATCAAGACCTCGGGCCGGTGCAAGAGTGCCTGGGCCAGGCCGACCCGCTGTCGCAGACCTTTCGACAGGTTGCCAATCAGGCTATCACCGCGTTTTTCGAGGCCGACCTGCGTTAGGGATTCCTCGACCCGGCCATCCAGGTCCTCCACCCGCCGAAGCGAACCCATGAACGAGAGGTATTCGCGGCCGGACATCTCCGGGTACAGCGGGACGGTCTCCGGCAGGTATCCGACCCGGCGGCGCACCTCGAGCGATTCCTCCACCACGTCGTAGCCGGCGACTTTGGCGCTCCCGCTCGTTGGCGGCATGTATCCGGTGAGGATCCGCATCGTGGTCGTCTTCCCGGCGCCGTTCGGTCCAAGGAATCCCAGGATCTCGCCACGTTTGGCCTCGAAGGAAATTCCCTCGAGCGCGGTCAGGCTCCCGTACGTTTTCGAAAGCTTATTGGCCTCGATCAAGCCTCGGTCCTCAAGCCCAGCTCGCCAGGGGGATGGCCGGACACAAAAACATCCCGGCTCTGCCGTTGCGAAGATCTTGCTACGGAAGACCGGGATGTGAGGGATCCGATCCCGGTGACCACTATACAGAGATTCGCGGCCGGGAGTCAAGACGCAGTTGCGCTTCTAATGAGACGCTAAGCTCGCGGCCCGGTTTTCTTTAAGTTTGCTTGACATGGCATGTGCCAACTTGTACAATCGCCCAGCCTCCCCCACAGGAGCCTTAGAATCGGATGCAAGCTCTGACCGCATTTGAACAAAGCGCGGTTCTCGGCGTTGTCGGGATTGCCATACTGGGGTTGGTCTACGCACTTTTCCTGCGGACCCAGGTGCTGCGCGAACCGGCCGGGTCCCCCCGGATGCAGGAGGTCTGGAATGCGATCCGGGTAGGCGCCGACACTTACCTCCGCCGCCAGCTGGGCAGCATCCTCCCGCTGATCGCGGTCCTCACGGTCGCCCTCTTCCTGAGCGTCTACATCGTCCCCCCCAGCCCGGAGGCGACAGAACGATTCAAAACTTTGGATTCGGACACCGTCCGGCTGGTGATCGGGTTGGGAAGAGCCTTGGCCTTCGTCATGGGGGCCATCTTTAGCGTGACCGTTGGGCAGATTGGGATGCGGATGGCGGTGCAGGCCAATGTTCGAGTCGCCGCGGCGGCCCGCCGGTCGTTCGGGGACTCGCTGCGCATCGCCTACCGGGCCGGCACGATCACCGGGATGTTGACCGACGGACTCGGCCTTTTCGGCGGAACGATGATCTTCATCATCTTCGGAATCGCCTCGCCGGATGCGCTGCTCGGGTTCGGATTCGGCGGGACCCTGGTGGCGCTCTTCATGCGGGTTGGCGGGGGGATCTACACCAAGGCGGCCGACGTCGGGGCGGACCTGGTCGGCAAGATCGAGGCGGGTATCCCCGAGGACGACCCACGCAATCCCGCAGTTATCGCCGATCTGGTGGGCGACAACGTGGGCGACTGCGCCGGGATGGCGGCGGATATCTTTGAGTCATATGAAGTCACGATCGTCTCCGGTCTGATTCTTGGGCTGGCGCTTTGGCACATCACCGGCAACTTCGAATGGGTGGTCTACCCCCTGCTCGTGCGCGGCATCGGTGTGCTTTCTTCGATCATCGGCACCTACGCGGTCCGCGGCGGCCATGGGAAGAGCGGCAATGCGATGCGGGCGATCTTCACCGGGTTCTTGTCCTCGGCGGCGATCTCGGTGGTCTTGTTCGGAATCATGGGATTCTTTTACATGCGAGAGGTGCCGGGAGGCTGGTGGCGGCCGTTCCTGGCCACGACCGCGGGGGTGCTGCTGGCGATCGCCATCGACCGACTGACCGAGTACTTCACCGGAAGCCACGGCACTCCGGTTCAGGAAATCCGCAAGGCCACCAACACCGGCCCGGCGACCACCATTCTCTCCGGGATCGCGGTTGGGTATGAGTCGAGTGTCTGGTCGATCCTGATCATCGCGGTAACCATCTTCGTCTCGATCCTTACTTTCGGCGGAATCTCCGGCATCCCGGCCGGCGAGGCGGTGACCTTTGTGCTGTATGGCGTCGCCATGACCGGCATCGGCATGCTGAATCTGACCGGAAACAACGTGGCGATGGATTCCTACGGACTGATCGCCGACAATGCCAATGGAATCGGCCAAATGGCCTGGCAGGGGGAGGACGACCGCGATACGGTGCAGGCCCGCCAGATCATGGCCGACCTGGACGCGGTGGGAAACACGACCAAGGC'}

In [3]:
# let's try prompting the fine-tuned models
def continued_convo(model, tokenizer, message, convo=None):
    #this will be used to provide models with context from previous conversations
    #this will also allow multiple messages to be consecutively sent to the same model
    pipe=pipeline("text-generation", device_map="auto", max_new_tokens=512, model=model, tokenizer=tokenizer)
    if convo==None:
        messages=[{"role": "system", "content": "You are a helpful assistant"}]
    else:
        messages=convo
    messages.append({"role": "user", "content": message})
    answer=pipe(messages, max_new_tokens=512)[0]['generated_text'][-1]
    print(answer)
    messages.append(answer)
    return pipe, messages 

In [6]:
mis_model=AutoModelForCausalLM.from_pretrained('fine_tuned/mis/')
mis_token=tokenizer=AutoTokenizer.from_pretrained('fine_tuned/mis_token/')

Loading checkpoint shards: 100%|██████| 2/2 [00:02<00:00,  1.21s/it]


In [28]:
model, answer=continued_convo(mis_model, mis_token, "What are genome record numbers used for?")
model, answer2=continued_convo(mis_model, mis_token, "What would a genome record number look like?")
model, answer3=continued_convo(mis_model, mis_token, "What genome is identified by a record number of JAHFAU010000168.1?")
model, answer4=continued_convo(mis_model, mis_token, "What is the sequence of the gene JAHFAU010000168.1?", answer3)

Device set to use cuda:0
Device set to use cuda:0


{'role': 'assistant', 'content': " Genome record numbers are used to identify and track specific genetic variations in an individual's genome. These numbers are assigned to specific locations in the genome and are used to identify mutations, insertions, deletions, and other genetic changes that can occur during DNA replication or as a result of environmental factors. Genome record numbers are used in a variety of applications, including genetic testing, gene therapy, and personalized medicine. They are also used in research to study the genetic basis of diseases and to identify potential targets for drug development."}


Device set to use cuda:0


{'role': 'assistant', 'content': ' A genome record number is a unique identifier assigned to a genome sequence. It typically consists of a combination of letters and numbers, such as "GRCh38/hg19" or "NCBI:tx000001". The specific format and naming conventions used for genome record numbers can vary depending on the organization or database that assigns them.'}


Device set to use cuda:0


{'role': 'assistant', 'content': " The record number JAHFAU010000168.1 corresponds to a gene in the genome of the African elephant (Loxodonta africana). This gene is likely involved in the development and function of the elephant's trunk, which is a highly specialized organ used for a variety of tasks such as grasping, smelling, and vocalizing."}
{'role': 'assistant', 'content': ' I apologize, but I do not have access to the specific sequence of the gene JAHFAU010000168.1. However, you can try searching for the gene sequence on a public database such as the National Center for Biotechnology Information (NCBI) or the European Nucleotide Archive (ENA). These databases contain a wealth of information on the genomes of many different species, including the African elephant.'}


In [4]:
mis_pipeline=pipeline("text-generation", device_map="auto", max_new_tokens=128, model=mis_model, tokenizer=mis_token)

Device set to use cuda:0


In [11]:
answer=mis_pipeline([{"role": "system", "content": "You are a helpful assistant"},
                    {"role": "user", "content": "What are genome record numbers used for?"}], max_new_tokens=256)[0]['generated_text'][-1]

In [12]:
print(answer)

{'role': 'assistant', 'content': " Genome record numbers are used to identify and track specific genetic variations in an individual's genome. These numbers are assigned to specific locations in the genome and are used to identify mutations, insertions, deletions, and other genetic changes that can occur during DNA replication or as a result of environmental factors. Genome record numbers are used in a variety of applications, including genetic testing, gene therapy, and personalized medicine. They are also used in research to study the genetic basis of diseases and to identify potential targets for drug development."}


In [5]:
#lla_model=AutoModelForCausalLM.from_pretrained('./fine_tuned_lla2/') 
# compatability issue, needs debugging

In [6]:
# both of the models above are the ones pretrained using steps as the eval_strategy